In [ ]:
# GET /hello_world

import json
print(json.dumps({
  'hello': 'world'
}))

## Template for event trigger to ETL data

In [ ]:
# POST /handle_event

GRAPHQL_ENDPOINT = ""
ADMIN_SECRET = ""

import json
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

def handle_insert(row, client):
    id = int(row['id'])
    name = str(row['name'])
    description = str(row['description'])
    # In reality you would follow the URL from row['url']
    content = "dummy content"
    gql_query = gql("""
            mutation insertItem($id: int!, $name: text!, $description: text!) {
                insert_Product_one(object: { product_id: $id, name: $name, description: $description }) {
                    id
                }
            }
        """)
    print(client.execute(gql_query, variable_values={
        'id': id, 'name': name, 'description': description}))


def handle_delete(row, client):
    id = int(row['id'])
    gql_query = gql("""
            mutation deleteItem($id: int!) {
                delete_Product(where: {product_id: { _eq: $id } }) {
                    affected_rows
                }
            }
        """)
    print(client.execute(gql_query, variable_values={
        'id': id}))


def handle_event(request):
    gql_headers = {'x-hasura-admin-secret': ADMIN_SECRET}
    # Create a GraphQL client with the request transport
    transport = RequestsHTTPTransport(
        url=GRAPHQL_ENDPOINT, headers=gql_headers)
    client = Client(transport=transport)

    event = request['body']['event']
    op = event['op']
    if op == 'INSERT':
        row = event['data']['new']
        handle_insert(row, client)
    elif op == 'UPDATE':
        old_row = event['data']['old']
        new_row = event['data']['new']
        # TODO: Do something
    elif op == 'DELETE':
        old_row = event['data']['old']
        handle_delete(old_row, client)
    else:
        print(str(event))
    return request

print(json.dumps(handle_event(json.loads(REQUEST))))

In [ ]:
# ResponseInfo POST /handle_event
print(json.dumps({
    "headers" : {
        "Content-Type" : "application/json"
    },
    "status" : 200
}))

## Template to handle user's question by querying Weaviate + OpenAI

In [ ]:
# POST /handle_query

GRAPHQL_ENDPOINT = ""
ADMIN_SECRET = ""
OPENAI_API_KEY = ""

import json
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

def handle_query(request):
    user_query = request['body']['input']['user_query']

    gql_headers = dict()
    gql_headers['x-hasura-admin-secret'] = ADMIN_SECRET

    # Create a GraphQL client with the request transport
    transport = RequestsHTTPTransport(
        url=GRAPHQL_ENDPOINT, headers=gql_headers)
    client = Client(transport=transport)

    # Send the GraphQL request
    gql_query = gql("""
            query getProducts($user_query: text!) {
                Product(where: { vector: { near_text: $user_query}}, limit: 10) {
                    name
                    description
                }
            }
        """)
    result = client.execute(gql_query, variable_values={
                            'user_query': user_query})
    prompt = """
    Use the following pieces of context to answer the question at the end.
    If you don't know the answer, just say that you don't know, don't try to make up an answer.
    \n
    """
    prompt += user_query

    resumes = result["Product"]

    for resume in resumes:
        prompt += "Product Name:"
        prompt += resume["name"]
        prompt += "Product Description: "
        prompt += resume["description"]
        prompt += "\n"

    prompt += "\nQuestion: {question}"


    llm = OpenAI(model="text-davinci-003",
                openai_api_key=OPENAI_API_KEY)
    chain = LLMChain(llm=llm, prompt=PromptTemplate.from_template(prompt))

    return chain.run({"question":user_query})

print(json.dumps(handle_query(json.loads(REQUEST))))

In [ ]:
# ResponseInfo POST /handle_query
print(json.dumps({
    "headers" : {
        "Content-Type" : "application/json"
    },
    "status" : 200
}))